In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
from nltk.corpus import stopwords

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [6]:
# Some data structures that will be useful
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [7]:
##################################################
# Read prediction                                #
##################################################
# From baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

In [6]:
# Generate a negative set

userSet = set()
bookSet = set()
readSet = set()

for u,b,r in allRatings:
    userSet.add(u)
    bookSet.add(b)
    readSet.add((u,b))

lUserSet = list(userSet)
lBookSet = list(bookSet)

notRead = set()
# For each (user,book) entry in the validation set, sample a
# negative entry by randomly choosing a book that user hasn’t read
for u,b,r in ratingsValid:
    b = random.choice(lBookSet)
    while ((u,b) in readSet or (u,b) in notRead):
        b = random.choice(lBookSet)
    notRead.add((u,b))

readValid = set()
for u,b,r in ratingsValid:
    readValid.add((u,b))
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom > 0:
        return numer/denom
    return 0

In [167]:
# # ratingsPerUser[u].append((b,r))
# # ratingsPerItem[b].append((u,r))
# # for each sample entry, get the books read by that user u
# # for each user u2 that has rated the same book,
#     # calculate the similarity: between the books u has rated and
#     # the books that u1 has rated
#     # the idea here is that if user u and u1 tend to read the same books,
#     # then u will also read b
#     # if there is sufficient similarity, decide that user u will read b

# correct = 0
# # make predictions on the validation set XValid and compare to yvalid
# for (label,sample) in [(1, readValid), (0, notRead)]:
#     for (u,b) in sample:
#         maxItemSim = 0
#         avgItemRating = 0
#         books = set(ratingsPerUser[u])
#         # Jaccard item-to-item similarity
#         for u2,r in ratingsPerItem[b]:
#             sim = Jaccard(books, set(ratingsPerUser[u2]))
#             if sim > maxItemSim:
#                 maxItemSim = sim
#             avgItemRating += r
        
#         avgItemRating = 0 if len(ratingsPerItem[b]) == 0 else avgItemRating/len(ratingsPerItem[b])
        
#         maxUserSim = 0
#         avgUserRating = 0
#         users = set(ratingsPerItem[b])
#         # Jaccard user-to-user similarity
#         for b2,r in ratingsPerUser[u]:
#             sim = Jaccard(users,set(ratingsPerItem[b2]))
#             if sim > maxUserSim:
#                 maxUserSim = sim
#                 avgUserRating += r
#         avgUserRating = 0 if len(ratingsPerUser[u]) == 0 else avgUserRating/len(ratingsPerUser[u])
                
#         pred = 0
#         # prediction based on Jaccard threshold and popularity threshold
#         if maxUserSim > 0.013 or maxItemSim > 0.2 or len(ratingsPerItem[b]) > 40:
#             pred = 1
#         # if avgItemRating is above a threshold
#         # if avgUserRating is within range [avgItemRating-.5, avgItemRating+.5]
#         if avgItemRating > 3.5 and len(ratingsPerItem[b]) > 20:
#             pred = 1
#         if avgItemRating-.3 <= avgUserRating <= avgItemRating+.3:
#             pred = 1
#         if pred == label:
#             correct += 1
# correct / (len(readValid) + len(notRead))
# # 0.74820 for no maxItemSim
# # 0.73210 for maxItemSim > 0.013
# # 0.74190 for maxItemSim > 0.053
# # 0.74395 for maxItemSim > 0.063
# # 0.74585 for maxItemSim > 0.073
# # 0.74650 for maxItemSim > 0.083
# # 0.74755 for maxItemSim > 0.1
# # 0.74820 for maxItemSim > 0.15
# # 0.74830 for maxItemSim > 0.2
# # 0.74820 for maxItemSim > 0.25

# # 0.74805 avgItemRating-.5 <= avgUserRating <= avgItemRating+.5, maxItemSim > 0.25
# # 0.74785 avgItemRating-.8 <= avgUserRating <= avgItemRating+.8, maxItemSim > 0.25
# # 0.74830 avgItemRating-.4 <= avgUserRating <= avgItemRating+.4, maxItemSim > 0.25
# # 0.74840 avgItemRating-.4 <= avgUserRating <= avgItemRating+.4, maxItemSim > 0.2
# # 0.74830 avgItemRating-.3 <= avgUserRating <= avgItemRating+.3, maxItemSim > 0.2
# # 0.74835 avgItemRating > 3.5 and len(ratingsPerItem[b]) > 20, maxItemSim > 0.25
# # 0.74840 avgItemRating > 3.5 and len(ratingsPerItem[b]) > 20, maxItemSim > 0.2
# # 0.7485 avgItemRating > 3.5 and len(ratingsPerItem[b]) > 20, avgItemRating-.4 <= avgUserRating <= avgItemRating+.4:
# # 0.7485 avgItemRating > 3.5 and len(ratingsPerItem[b]) > 20, avgItemRating-.3 <= avgUserRating <= avgItemRating+.3

0.7485

In [179]:
# Question 5
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    maxItemSim = 0
    avgItemRating = 0
    books = set(ratingsPerUser[u])
    # Jaccard item-to-item similarity
    for u2,r in ratingsPerItem[b]:
        sim = Jaccard(books, set(ratingsPerUser[u2]))
        if sim > maxItemSim:
            maxItemSim = sim
        avgItemRating += r
    avgItemRating = 0 if len(ratingsPerItem[b]) == 0 else avgItemRating/len(ratingsPerItem[b])
    
    maxUserSim = 0
    avgUserRating = 0
    users = set(ratingsPerItem[b])
    # Jaccard user-to-user similarity
    for b2,r in ratingsPerUser[u]:
        sim = Jaccard(users,set(ratingsPerItem[b2]))
        if sim > maxUserSim:
            maxUserSim = sim
            avgUserRating += r
    avgUserRating = 0 if len(ratingsPerUser[u]) == 0 else avgUserRating/len(ratingsPerUser[u])
    
    pred = 0
    if maxUserSim > 0.02 or maxItemSim > 0.2 or len(ratingsPerItem[b]) > 35:
        pred = 1
#     # if avgItemRating is above a threshold 
#     # and the item has a sufficient number of ratings to source a reliable average
#     if avgItemRating > 3.5 and len(ratingsPerItem[b]) > 20:
#         pred = 1
#     # if avgUserRating is within range [avgItemRating-.3, avgItemRating+.3]
#     if avgItemRating-.3 <= avgUserRating <= avgItemRating+.3:
#         pred = 1
        
    _ = predictions.write(u + ',' + b + ',' + str(pred) + '\n')

predictions.close()

# 0.7683 len(ratingsPerItem[b]) > 35
# 0.7648 baseline
# 0.769 maxUserSim > 0.02 or maxItemSim > 0.2 or len(ratingsPerItem[b]) > 3

In [143]:
# # Generate a negative set for test set and validation set

# userSet = set()
# bookSet = set()
# readSet = set()

# for u,b,r in allRatings:
#     userSet.add(u)
#     bookSet.add(b)
#     readSet.add((u,b))

# lUserSet = list(userSet)
# lBookSet = list(bookSet)

# notReadAll = set()
# # For each (user,book) entry in the validation set, sample a
# # negative entry by randomly choosing a book that user hasn’t read
# for u,b,r in ratingsValid:
#     b = random.choice(lBookSet)
#     while ((u,b) in readSet or (u,b) in notReadAll):
#         b = random.choice(lBookSet)
#     notReadAll.add((u,b))

# readValid = set()
# for u,b,r in ratingsValid:
#     readValid.add((u,b))
    
# # add negative set to validation set
# Xvalid = list(readValid) + list(notReadAll)
# yvalid = [1]*len(readValid) + [0]*len(notReadAll)
# # print(Xvalid[len(ratingsValid):len(ratingsValid)+5])
# # print(yvalid[len(ratingsValid):len(ratingsValid)+5])

# notReadTrain = set()
# for u,b,r in ratingsTrain:
#     b = random.choice(lBookSet)
#     while ((u,b) in readSet or (u,b) in notReadAll):
#         b = random.choice(lBookSet)
#     notReadAll.add((u,b))
#     notReadTrain.add((u,b))
    
# readTrain = set()
# for u,b,r in ratingsTrain:
#     readTrain.add((u,b))
    
# # add negative sets to corresponding sets
# Xtrain = list(readTrain) + list(notReadTrain)
# ytrain = [1]*len(readTrain) + [0]*len(notReadTrain)

In [146]:
# def feature(datum):
#     u, b = datum
# #     maxItemSim = 0
# #     minItemSim = 1
# #     books = set(ratingsPerUser[u])
# #     # Jaccard item-to-item similarity
# #     for u2,r in ratingsPerItem[b]:
# #         if u2 == u:
# #             continue
# #         sim = Jaccard(books, set(ratingsPerUser[u2]))
# #         if sim > maxItemSim:
# #             maxItemSim = sim
# # #         if sim < minItemSim:
# # #             minItemSim = sim
#     avgItemRating = 0
#     for u2,r in ratingsPerItem[b]:
#         avgItemRating += r
        
#     avgItemRating = 0 if len(ratingsPerItem[b]) == 0 else avgItemRating/len(ratingsPerItem[b])
    
#     maxUserSim = 0
# #     minUserSim = 1
#     avgUserRating = 0
#     users = set(ratingsPerItem[b])
#     # Jaccard user-to-user similarity
#     for b2,r in ratingsPerUser[u]:
#         if b2 == b:
#             continue
#         sim = Jaccard(users,set(ratingsPerItem[b2]))
#         if sim > maxUserSim:
#             maxUserSim = sim
# #         if sim < minUserSim:
# #             minUserSim = sim
#         avgUserRating += r
#     avgUserRating = 0 if len(ratingsPerUser[u]) == 0 else avgUserRating/len(ratingsPerUser[u])

#     bookPopularity = len(ratingsPerItem[b])
#     avgRating = 0    
#     feat = [1]
#     feat.append(bookPopularity)
#     feat.append(avgItemRating)
# #     feat.append(avgUserRating)
# #     feat.append(maxItemSim)
#     feat.append(maxUserSim)
#     return feat

# # X = [feature(d) for d in data]
# # y = [1]*len(X)
# fitXtrain = [feature(d) for d in Xtrain]
# mod = linear_model.LogisticRegression(C=10**(-3))
# mod.fit(fitXtrain, ytrain)
# fitXvalid = [feature(d) for d in Xvalid]
# pred = mod.predict(fitXvalid)
# correct = pred == yvalid
# sum(correct) / len(correct)
# # 0.7097 class_weight='balanced' pop, maxUserSim 
# # 0.65235 class_weight='balanced' pop, maxUserSim, maxItemSim 
# # 0.65235 class_weight='balanced' pop, maxUserSim, maxItemSim, minUser/Item 
# # adding min doesnt change anything since its always 0
# # 0.6559 class_weight='balanced' pop, maxItemSim 
# # 0.63225 class_weight='balanced' pop, maxUserSim, maxItemSim avgItem/UserSim 
# # 0.72595 C=10**(-4), class_weight='balanced' pop, maxItem, maxUser 
# # 0.72715 C=10**(-4), class_weight='balanced' pop, maxUser

# # 0.72725 C=10**(-3), class_weight='balanced' pop, maxUser
# # 0.72725 C=10**(-3) pop, maxUser
# # 0.72770 C=10**(-3) pop, maxUser, avgItemRating
# # 0.7048 C=10**(1) pop, maxUser
# # 0.7041 C=10**(2) pop, maxUser
# # 0.70375 C=10**(3) pop, maxUser
# # 0.72675 C=10**(-3) pop, maxUser, avgItemRating
# # 0.68685 C=10**(-3) pop, avgItem/UserRating, maxUser

0.7277

In [176]:
# theta = mod.coef_
# theta

In [147]:
# Xtest = []
# predictions = open("predictions_Read.csv", 'w')
# for l in open("pairs_Read.csv"):
#     if l.startswith("userID"):
#         predictions.write(l)
#         continue
#     u,b = l.strip().split(',')
#     Xtest.append((u,b))
    
# fitXtest = [feature(d) for d in Xtest]
# predict = mod.predict(fitXtest)
# for i in range(len(predict)):
#     u,b = Xtest[i]
#     pred = predict[i]
#     _ = predictions.write(u + ',' + b + ',' + str(pred) + '\n')

# predictions.close()

In [122]:
# data = list(zip([1]*len(readValid), readValid)) + list(zip([0]*len(notRead), notRead))
# print((data))

In [3]:
##################################################
# Category prediction (CSE158 only)              #
##################################################
data = []

for d in readGz("train_Category.json.gz"):
    data.append(d)
# data[0]
# {'user_id': 'u75242413',
#  'review_id': 'r45843137',
#  'rating': 4,
#  'review_text': "a clever book with a deeply troubling premise and an intriguing protagonist. Thompson's clean, sparse prose style kept each page feeling light even as some rather heavy existential questions dropped upon them. I enjoyed it. \n and that cover design is boom-pow gorgeous.",
#  'n_votes': 1,
#  'genre': 'mystery_thriller_crime',
#  'genreID': 3}

In [4]:
genresPerUser = {}
# avgGenresRatingPerUser = {}
# globalGenreAvg = [0, 0, 0, 0, 0]
# totalReviews = [0, 0, 0, 0, 0]
for d in data:
    u = d['user_id']
    gid = d['genreID']
    if u not in genresPerUser:
        genresPerUser[u] = [0]*5
#     if u not in avgGenresRatingPerUser:
#         avgGenresRatingPerUser[u] = [0]*5
    genresPerUser[u][gid] += 1
#     avgGenresRatingPerUser[u][gid] += d['rating']
#     globalGenreAvg[gid] += d['rating']
#     totalReviews[gid] += 1
# # compute user averages
# for u in genresPerUser.keys():
#     for i in range(0,5):
#         if genresPerUser[u][i] == 0:
#             avgGenresRatingPerUser[u][i] = 0
#         else:
#             avgGenresRatingPerUser[u][i] /= genresPerUser[u][i]
# # compute global averages            
# for i in range(0,5):
#     if totalReviews[i] == 0:
#         globalGenreAvg[i] = 0
#     else:
#         globalGenreAvg[i] /= totalReviews[i]

In [7]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stop_words = set(stopwords.words('english'))

for d in data:
  r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
  for w in r.split():
    if w not in stop_words:
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
# counts[:10]
# [(739469, 'the'),
#  (447352, 'and'),
#  (393557, 'a'),
#  (373089, 'to'),
#  (364764, 'i'),
#  (324675, 'of'),
#  (218882, 'is'),
#  (212488, 'in'),
#  (204960, 'it'),
#  (193791, 'this')]

In [5]:
def feature1(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    for w in r.split():
        if w in wordSet:
            feat[wordId[w]] += 1
    u = datum['user_id']
    if u in genresPerUser:
        feat += (genresPerUser[u])
#         feat += avgGenresRatingPerUser[u]
    else:
        feat += [0,0,0,0,0]
        # replace with global average rating for each genre instead of 0
#         feat += globalGenreAvg
#         feat += [0,0,0,0,0]
    feat.append(1) #offset
    return feat


# userGenreCount: [_,_,_,_,_] num of ratings for each genre
# avgUserGenreRating: [_,_,_,_,_] user's avg rating for books in each genre

In [8]:
### Question 8

NW = 15000 # dictionary size
words = [x[1] for x in counts[:NW]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

X = [feature1(d) for d in data]
y = [d['genreID'] for d in data]

Xtrain = X[:9*len(X)//10]
ytrain = y[:9*len(y)//10]
Xvalid = X[9*len(X)//10:]
yvalid = y[9*len(y)//10:]

mod = linear_model.LogisticRegression(C=1)
mod.fit(Xtrain, ytrain)

pred = mod.predict(Xvalid)
correct = pred == yvalid
sum(correct) / len(correct)
# 0.5093 C=10**(-4)
# 0.6324 C=10**(-3)
# 0.6792 C=10**(-2)
# 0.6851 C=10**(-1)
# 0.6907 C=1
# 0.6862 C=10**(1)
# 0.6883 C=10**(2)
# 0.685 C=10**(3)
# 0.6886 C=10**(4)
# 0.823 with genre count, c=1, nw =2000
# 0.8259 with genre count, c=1, nw =1000 but poor on leaderboard
# 0.8242, 0.7088 leaderboard: genrecount, nw=3000, c=1
# 0.8738, 0.6714 leaderboard: genrecount avgrating nw=3000, c=1
# 0.7122 leaderboard: genrecount nw=4000, c=1
# 0.7148 leaderboard genrecount nw=5000, c=1
# 0.6634 leaderboard genrecount globalavg nw=5000, c=1
# 0.7194 leaderboard genrecount nw=7000, c=1
# 0.7314 leaderboard genrecount nw=10000, c=1
# 0.7398 leaderboard genrecount nw=15000, c=1

c:\users\davj5\miniconda3\envs\mae146\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.8336

In [10]:
# run on test set
data_test = []

for d in readGz("test_Category.json.gz"):
    data_test.append(d)
Xtest = [feature1(d) for d in data_test]
pred_test = mod.predict(Xtest)
predictions = open("predictions_Category.csv", 'w')
pos = 0

for l in open("pairs_Category.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    _ = predictions.write(u + ',' + b + ',' + str(pred_test[pos]) + '\n')
    pos += 1

predictions.close()